In [ ]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Pkg
Pkg.activate(".")
using Test
using LifeInsuranceDataModel
using BitemporalPostgres
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON


In [ ]:
if (haskey(ENV, "GITPOD_REPO_ROOT"))
    run(```psql -f sqlsnippets/droptables.sql```)
else
    ENV["SEARCHLIGHT_USERNAME"] = "bitemporalpostgres"
    ENV["SEARCHLIGHT_PASSWORD"] = "jw8s0F49KL"
    println("execute this in Your REPL")
    println("""run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)""")
end

LifeInsuranceDataModel.connect()

In [ ]:
c = find(Contract, SQLWhereExpression("id=?", 1))[1]
crOld = find(ContractRevision, SQLWhereExpression("ref_component=? and ref_invalidfrom=?", 1,9223372036854775807))[1]

cr =  ContractRevision(ref_component=c.id, description="mutation for testing active transactions")
w0 = Workflow(
    ref_history=c.ref_history,
    tsw_validfrom=ZonedDateTime(2022, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w0)
update_component!(crOld, cr, w0)
# commit_workflow!(w2)

In [ ]:
w=find(Workflow,SQLWhereExpression("id=?", 13))[1]
rollback_workflow!(w)

In [ ]:
function findversion(ref_history::DbId, tsdb::ZonedDateTime, tsw::ZonedDateTime)
    find(
        ValidityInterval,
        SQLWhereExpression(
            "ref_history=? and tsrworld @> TIMESTAMPTZ ? AND tsrdb @> TIMESTAMPTZ ?",
            ref_history,
            SQLInput(tsw),
            SQLInput(tsdb),
        ),
    )
end

findversion(DbId(9), now(tz"UTC"), now(tz"UTC"))